In [2]:
这是版本1 ;
问题在于 ;
1. 我只利用了 tf-idf 特征 ; --> bert;
2. 尽管利用了邻居和自己，但是邻居的定义存在问题(hard rank);--> 可以用我的weighted * mask;
3. 损失函数是纯监督 --> 其实可以纳入负采样无监督；

SyntaxError: invalid syntax (<ipython-input-2-2abcc2124176>, line 3)

In [3]:
def result(G, output, label_idx, idx2label):
    SF = nn.Softmax(dim=-1)
    score, pred = SF(output).max(dim=-1)
    sms = [G.node[i]['text'] for i in label_idx]
    df = pd.DataFrame()
    df['sms']= sms
    df['score']=score.data
    df['pred']=[idx2label[i] for i in pred.numpy()]
    df['label']=[g4.node[i]['label'] for i in label_idx]
    return df

def get_clf_prediction(model, X_word, X_char, y):
    print('X shape:{}, y_shape:{}'.format(len(X_word), len(y)))
    y_pred = model.predict([X_word,
                            np.array(X_char).reshape((len(X_char),
                                                         max_len, max_len_char))])
    pred = [idx2label[i] for i in np.argmax(y_pred,axis=-1)]
    true = [idx2label[i] for i in y]
    return true, pred

def get_pr(Res, cls):
    #某类 正确识别数量/该类 总识别数量
    tmp  = Res[Res.true == cls]
    a = sum(tmp.true.values == tmp.pred.values)
    b = Res[Res.pred == cls].shape[0]
    pr = a/b
    return pr

def get_rc(Res, cls):
    #某类 正确识别数量/该类 总数量
    tmp  = Res[Res.true == cls]
    support = tmp.shape[0]
    a = sum(tmp.true.values == tmp.pred.values)
    b = Res[Res.true == cls].shape[0]
    rc = a/b
    return rc,support

def get_f1(pr, rc):
    # f1 = (2*pr*rc)/(pr+rc)
    f1 = (2*pr*rc)/(pr+rc)
    return f1

def evaluate(Res, cls):
    pr = get_pr(Res, cls)
    rc, support = get_rc(Res, cls)
    f1 = get_f1(pr, rc)
    return [cls, pr, rc, f1, support]

def Final_evalu(z):
    Res = pd.DataFrame(columns=['true', 'pred'])
    true, pred = z.label.tolist(), z.pred.tolist()
    Res['true'] = true
    Res['pred'] = pred
    entitys = Res.true.unique()
    records = []
    for i in entitys:
        tmp = evaluate(Res, i)
        records.append(tmp)
    record = pd.DataFrame.from_records(records)
    record.columns =['cls','精确率','召回率', 'F1', 'support']
    record = record.set_index('cls')
    record = record.sort_index()   
    return record


In [4]:
import networkx as nx
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
import torch
import torch.nn as nn
import copy
# from graphviz import Digraph
from torch.nn import init
from torch.autograd import Variable
import torch.nn.functional as F
import time
import random
from sklearn.metrics import f1_score
# from collections import defaultdict
# from ClfTemplates import get_all_template_path, sample_template, sample_template_path
# from ClfUse import showClfResult, get_samples, labeledDataFromJson
# from Ner_model import get_X_Y, lstm_crf, predict, preprocess
# from Ner_data_make import create_fake_label, fake2real
from keras.preprocessing.sequence import pad_sequences

from keras.models import Model, Input, Sequential
from keras.layers import LSTM, Embedding, SpatialDropout1D,concatenate, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss
from keras.utils import to_categorical
import keras
from collections import Iterable 
import pandas as pd
import numpy as np
import random
import re
import warnings
warnings.filterwarnings("ignore")


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [5]:
def get_cos_similarity(sms, templates):
    '''计算一条新sms 与 每个 template 相似度;'''
    def cos_sim(a, b):
        return dot(a, b) / (norm(a) * norm(b))
    return [cos_sim(i, sms) for i in templates]


def tfIdfVector(corpus):
    '''
    corpus is a list of sentences:
    ['This is an example', 'hello world', ...]
    '''
    vectorizer = CountVectorizer()
    transformer = TfidfTransformer()
    x = vectorizer.fit_transform(corpus)
    tfidf = transformer.fit_transform(x)
    return tfidf.toarray()

def GraphBuild(template_df, new_sms, K=10, thresh=None):
    '''
    之前版本没考虑到采样需要全图遍历；故在入图过程就把K_nerghs当作点属性加进去；
    new_sms: test samples, df type;
    
    '''
    def findNeighsK(arr, K):
        K+=1 #入图时neighs 不包括自己，故这里先加一个保证去除自己之后仍然有K neighs;
        idx=np.argpartition(arr, -K)[-K:]
        return idx

    corpus, labels = template_df.sms.tolist(), template_df.cls.tolist()
    new_labels = new_sms.cls.tolist()
    new_sms = new_sms.sms.tolist()
    num_sms = len(new_sms)
    for single in new_sms:
        corpus.append(single.lower())
    for single in new_labels:
        labels.append(single)
    all_tfidf = tfIdfVector(corpus)
    adj_mat = cosine_similarity(all_tfidf)
    print('adj mat shape:{}'.format(adj_mat.shape))
    G = nx.Graph()
    G.add_nodes_from(range(len(adj_mat)))
    for i in range(len(G.nodes())):
        G.node[i]['vec'] =  all_tfidf[i]
        G.node[i]['text'] = corpus[i]
        G.node[i]['label'] = labels[i]
        G.node[i]['neighs_k'] = set(findNeighsK(adj_mat[i], K)) - set([i])
    return G

In [14]:
gsm_templates_df = pd.read_csv('gsm_templates_df.csv')
gsm_templates_df.dropna(inplace=True)
gsm_templates_df.columns = ['sms', 'cls']
test_df = gsm_templates_df.sample(100)
train_df = gsm_templates_df[~gsm_templates_df.index.isin(test_df.index)]

In [15]:
g4 = GraphBuild(train_df, test_df)

adj mat shape:(1195, 1195)


In [16]:
"""
Set of modules for aggregating embeddings of neighbors.
"""
class MeanAggregator_QBv2(nn.Module):
    """
    adjMat@embMat: [bs, uniqueNode]@[uniqueNode, F]=[bs, F]
    
    要改，之前没注意每次采样全图遍历；
    如无必要，不要让一个实例挂一个毒瘤: G (20G内存)
    """
    def __init__(self, features, cuda=False, gcn=False): 
        """
        Initializes the aggregator for a specific graph.
        features -- function mapping LongTensor of node ids to FloatTensor of feature values.
        cuda -- whether to use GPU;
        gcn --- whether to perform concatenation GraphSAGE-style, or add self-loops GCN-style
        """
        super(MeanAggregator_QBv2, self).__init__()
        self.features = features
        self.cuda = cuda
        self.gcn = gcn
        
    def forward(self, nodes, to_neighs, num_sample=10):
        """
        nodes --- list of nodes in a batch, [bs]
        to_neighs --- list of sets, 
                      each set is the set of neighbors for node in batch
        num_sample --- number of neighbors to sample. 
        """
        _set = set
        if not num_sample is None:
            _sample = random.sample
            samp_neighs = [_set(_sample(to_neigh, num_sample))
                           if len(to_neigh) >= num_sample else to_neigh 
                           for to_neigh in to_neighs]
        else:
            samp_neighs = to_neighs
    
        # 纳入target idx;
        if self.gcn:
            samp_neighs = [set.union(samp_neigh, set([nodes[i]])) for i, samp_neigh in enumerate(samp_neighs)]
        unique_nodes_list = list(set.union(*samp_neighs))
        unique_nodes = {n:i for i,n in enumerate(unique_nodes_list)}
        mask = Variable(torch.zeros(len(samp_neighs), len(unique_nodes)))
        column_indices = [unique_nodes[n] for samp_neigh in samp_neighs for n in samp_neigh]   
        row_indices = [i for i in range(len(samp_neighs)) for j in range(len(samp_neighs[i]))]
        mask[row_indices, column_indices] = 1
        if self.cuda:
            mask = mask.cuda()
        num_neigh = mask.sum(1, keepdim=True)
        mask = mask.div(num_neigh)
        if self.cuda:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list).cuda())
        else:
            embed_matrix = self.features(torch.LongTensor(unique_nodes_list))
        to_feats = mask.mm(embed_matrix)
        return to_feats

    
class Encoder_QBv2(nn.Module):
    """
    ([cls, 2f]@[2f, bs]).T = [bs, cls] 
    Encodes a node's using 'convolutional' GraphSage approach
    """
    def __init__(self, features, feature_dim, 
            embed_dim, adj_lists, aggregator,
            num_sample=10,
            base_model=None, gcn=False, cuda=False, 
            feature_transform=False): 
        super(Encoder_QBv2, self).__init__()

        self.features = features
        self.feat_dim = feature_dim
        self.adj_lists = adj_lists
        self.aggregator = aggregator
        self.num_sample = num_sample
        if base_model != None:
            self.base_model = base_model

        self.gcn = gcn
        self.embed_dim = embed_dim
        self.cuda = cuda
        self.aggregator.cuda = cuda
        self.weight = nn.Parameter(
                torch.FloatTensor(embed_dim, self.feat_dim if self.gcn else 2 * self.feat_dim))
        init.xavier_uniform(self.weight)


    def forward(self, nodes):
        """
        Generates embeddings for a batch of nodes.
        nodes     -- list of nodes; [bs]
        operations: 已知self_feat: [bs, F]
                    再agg_feat: [bs, F]
                    然后concat([agg_feat, self_feat]): [bs, 2F]
                    然后W:[F2, 2F] 
                    relu(W @[bs,2F].T) : [F2, bs] 
        """
        neigh_feats = self.aggregator.forward(nodes, [self.adj_lists[int(node)] for node in nodes], 
                self.num_sample)
        if not self.gcn:
            if self.cuda:
                self_feats = self.features(torch.LongTensor(nodes).cuda())
            else:
                self_feats = self.features(torch.LongTensor(nodes))
            combined = torch.cat([self_feats, neigh_feats], dim=1)
        else:
            combined = neigh_feats
        combined = F.relu(self.weight.mm(combined.t()))
        return combined



class SupervisedGraphSage_QB(nn.Module):
    def __init__(self, num_classes, enc):
        super(SupervisedGraphSage_QB, self).__init__()
        self.enc = enc
        self.weight = nn.Parameter(torch.FloatTensor(num_classes, enc.embed_dim))
        init.xavier_uniform(self.weight)
        self.loss = nn.NLLLoss(reduce=False)
        self.logsoftmax = nn.LogSoftmax(dim=-1)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, nodes, softmax=False):
        embeds = self.enc(nodes)
        scores = self.weight.mm(embeds)
        if softmax:
            return self.softmax(scores.t())
        return scores.t()

    def loss_softmax_mask(self, nodes, labels, mask=None):
        '''
        nodes shape: [N];
        labels shape: [N];
        mask shape: [N];
        '''
        if mask is None:
            mask = torch.ones(len(labels))
        logits = self.forward(nodes)
        A_soft = self.logsoftmax(logits)
        output = self.loss(A_soft, labels.squeeze())
        loss = output*mask.type(torch.float)
        loss = sum(loss)/sum(mask)
        return loss
    
    
def result(G, output, label_idx, idx2label):
    score, pred = output.max(dim=-1)
    sms = [G.node[i]['text'] for i in label_idx]
    df = pd.DataFrame()
    df['sms']= sms
    df['score']=score.data
    df['pred']=[idx2label[i] for i in pred.numpy()]
    df['label']=[g4.node[i]['label'] for i in label_idx]
    return df

In [17]:
def load_india_sms(G):
    feat_data = [G.node[i]['vec'] for i in G.nodes]
    labels = [G.node[i]['label'] for i in G.nodes]
    adj_lists = [G.node[i]['neighs_k'] for i in G.nodes]
    return feat_data, labels, adj_lists

In [18]:
np.random.seed(1)
random.seed(1)
feat_data, labels, adj_lists = load_india_sms(g4)
idx2label = {i:j for i, j in enumerate(np.unique(labels))}
label2idx = {j:i for i,j in idx2label.items()}
labels = np.array([label2idx[i] for i in labels])
num_cls = len(np.unique(labels))
features = nn.Embedding(len(feat_data), len(feat_data[0]))
features.weight = nn.Parameter(torch.FloatTensor(feat_data), requires_grad=False)
node_F = len(feat_data[0])
agg1 = MeanAggregator_QBv2(features, cuda=False, gcn=True)
enc1 = Encoder_QBv2(features, node_F, 128, adj_lists, agg1, num_sample=5, gcn=True, cuda=False)
agg2 = MeanAggregator_QBv2(lambda nodes : enc1(nodes).t(), cuda=False, gcn=True)
enc2 = Encoder_QBv2(lambda nodes : enc1(nodes).t(), 
               enc1.embed_dim, node_F, adj_lists, agg2, num_sample=5, gcn=True, cuda=False)

In [20]:
graphsage = SupervisedGraphSage_QB(num_cls, enc2)
#graphsage.cuda()
#rand_indices = np.random.permutation(len(node_list))
nodes = list(g4.nodes)
val = nodes[:50]
test_nodes = list(g4.nodes)[-test_df.shape[0]:]
train = nodes[50:-len(test_nodes)]
optimizer = torch.optim.SGD(filter(lambda p : p.requires_grad, graphsage.parameters()), lr = 0.8)
times = []
Loss = []
print('GO')
for batch in range(5000):
    batch_nodes2 = train[:200]
    batch_idx = copy.copy(train[:200])
    random.shuffle(train)
    start_time = time.time()
    optimizer.zero_grad()
    loss = graphsage.loss_softmax_mask(batch_nodes2,
            Variable(torch.tensor(labels[np.array(batch_idx)])))
    loss.backward()
    optimizer.step()
    end_time = time.time()
    times.append(end_time - start_time)
#     print(graphsage.forward(val, True)[0].sum())
    if batch%100 == 0:
        train_loss = graphsage.loss_softmax_mask(train,
            Variable(torch.tensor(labels[np.array(train)])))
        val_loss = graphsage.loss_softmax_mask(val,
            Variable(torch.tensor(labels[np.array(val)])))
        print('batch:{}; train_loss:{}; val_loss:{}'.format(batch, train_loss.data, val_loss.data))
    Loss.append(loss.data)

GO
batch:0; train_loss:2.8288917541503906; val_loss:2.82924747467041
batch:100; train_loss:1.558699369430542; val_loss:1.7151776552200317
batch:200; train_loss:1.4363244771957397; val_loss:1.6016473770141602
batch:300; train_loss:1.3642029762268066; val_loss:1.5427944660186768
batch:400; train_loss:1.3100275993347168; val_loss:1.4732929468154907
batch:500; train_loss:1.2747801542282104; val_loss:1.4334924221038818
batch:600; train_loss:1.2323875427246094; val_loss:1.4309425354003906
batch:700; train_loss:1.1975066661834717; val_loss:1.3915148973464966
batch:800; train_loss:1.166976809501648; val_loss:1.364182710647583
batch:900; train_loss:1.1433796882629395; val_loss:1.3438844680786133
batch:1000; train_loss:1.1128835678100586; val_loss:1.2868318557739258
batch:1100; train_loss:1.0935643911361694; val_loss:1.288069248199463
batch:1200; train_loss:1.0736873149871826; val_loss:1.278058409690857
batch:1300; train_loss:1.0496153831481934; val_loss:1.232122778892517
batch:1400; train_loss:

In [21]:
te_output = graphsage.forward(test_nodes)
test_res = result(g4, te_output, test_nodes, idx2label)
Final_evalu(test_res)

,精确率,召回率,F1,support
cls,,,,
sms_other,0.000000,0.000000,NaN,3
交易流水＿余额,NaN,0.000000,NaN,7
交易流水＿转账,0.769231,0.983607,0.863309,61
信用卡＿还款提醒,0.800000,1.000000,0.888889,8
信用卡＿逾期警告,NaN,0.000000,NaN,1
账号异常＿余额不足,0.250000,0.333333,0.285714,3
账号异常＿卡号冻结,NaN,0.000000,NaN,2
账号异常＿扣款失败,0.666667,0.500000,0.571429,8
贷前申请＿审核拒绝,NaN,0.000000,NaN,1


In [22]:
tra_output = graphsage.forward(train)
train_res = result(g4, tra_output, train, idx2label)
Final_evalu(train_res)

,精确率,召回率,F1,support
cls,,,,
sms_other,0.600000,0.078947,0.139535,38
交易流水＿余额,0.818182,0.176471,0.290323,51
交易流水＿转账,0.822850,0.962462,0.887197,666
信用卡＿申请失败,NaN,0.000000,NaN,3
信用卡＿还款提醒,0.619565,0.838235,0.712500,68
信用卡＿逾期警告,0.333333,0.052632,0.090909,19
账号异常＿余额不足,0.693548,0.716667,0.704918,60
账号异常＿信用额度不足,NaN,0.000000,NaN,1
账号异常＿卡号冻结,1.000000,0.210526,0.347826,19
